In [18]:
import pandas as pd

from scipy import stats

In [5]:
rna_seq_df = pd.read_csv('medians.csv',sep=',')
rna_seq_df

#The first row lists the cluster name (cluster_label), which matches the cell type alias shown in the Transcriptomic Explorer.
#The first column lists the unique gene identifier (gene), which in most cases is the gene symbol.

,Unnamed: 0,Lamp5_1,Lamp5_2,Lamp5_3,Lamp5_4,Lamp5_5,Lamp5_6,Lamp5 Lhx6_1,Lamp5 Lhx6_2,Lamp5 Lhx6_3,...,VLMC_2,VLMC_3,VLMC_4,VLMC_5,Endo_1,Endo_2,Endo_3,SMC,Macrophage_1,Macrophage_2
0,0610005C13Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
1,0610006L08Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,0610007P14Rik,6.600176,6.813337,6.704629,6.668049,6.773956,6.362829,6.691099,6.708878,7.162359,...,0.000000,6.974747,0.000000,5.633328,0.0,0.0,0.0,0.0,0.000000,0.0
3,0610009B22Rik,6.169217,6.473920,6.314496,6.533899,6.450335,6.585420,6.825147,6.757542,6.932700,...,0.313972,7.104814,6.486057,6.576098,0.0,0.0,0.0,0.0,5.304603,0.0
4,0610009E02Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
45764,n-R5s143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
45765,n-R5s144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
45766,n-R5s146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [6]:
list(rna_seq_df.columns)

['Unnamed: 0',
 'Lamp5_1',
 'Lamp5_2',
 'Lamp5_3',
 'Lamp5_4',
 'Lamp5_5',
 'Lamp5_6',
 'Lamp5 Lhx6_1',
 'Lamp5 Lhx6_2',
 'Lamp5 Lhx6_3',
 'Lamp5 Lhx6_4',
 'Lamp5 Lhx6_5',
 'Sncg_1',
 'Sncg_2',
 'Sncg_3',
 'Sncg_4',
 'Sncg_5',
 'Sncg_6',
 'Sncg_7',
 'Sncg_8',
 'Sncg/Ndnf HPF_1',
 'Sncg/Ndnf HPF_2',
 'Sncg/Ndnf HPF_3',
 'Sncg/Ndnf HPF_4',
 'Sncg/Ndnf HPF_5',
 'Sncg/Ndnf HPF_6',
 'Serpinf1_1',
 'Serpinf1_2',
 'Serpinf1_3',
 'Vip_1',
 'Vip_2',
 'Vip_3',
 'Vip_4',
 'Vip_5',
 'Vip_6',
 'Vip_7',
 'Vip_8',
 'Vip_9',
 'Vip_10',
 'Vip_11',
 'Vip_12',
 'Vip_13',
 'Vip_14',
 'Vip_15',
 'Vip_16',
 'Vip_17',
 'Sst Chodl_1',
 'Sst Chodl_2',
 'Sst_1',
 'Sst_2',
 'Sst_3',
 'Sst_4',
 'Sst_5',
 'Sst_6',
 'Sst_7',
 'Sst_8',
 'Sst_9',
 'Sst_10',
 'Sst_11',
 'Sst_12',
 'Sst_13',
 'Sst_14',
 'Sst_15',
 'Sst_16',
 'Sst_17',
 'Sst_18',
 'Sst_19',
 'Sst_20',
 'Sst_21',
 'Sst_22',
 'Sst_23',
 'Sst_24',
 'Sst_25',
 'Sst_26',
 'Sst_27',
 'Sst_28',
 'Sst_29',
 'Pvalb_1',
 'Pvalb_2',
 'Pvalb_3',
 'Pvalb_4',
 'Pvalb

Take first 93 columns - those are all the GABAergic cells according to https://celltypes.brain-map.org/rnaseq/mouse/cortex-and-hippocampus. GABAergic are inhibitory

In [7]:
gaba_cells = list(rna_seq_df.columns)[:92]
inhib_df = rna_seq_df[gaba_cells]
inhib_df

,Unnamed: 0,Lamp5_1,Lamp5_2,Lamp5_3,Lamp5_4,Lamp5_5,Lamp5_6,Lamp5 Lhx6_1,Lamp5 Lhx6_2,Lamp5 Lhx6_3,...,Pvalb_6,Pvalb_7,Pvalb_8,Pvalb_9,Pvalb_10,Pvalb_11,Pvalb_12,PIR Six3,Meis2_1,Meis2_2
0,0610005C13Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0610006L08Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0610007P14Rik,6.600176,6.813337,6.704629,6.668049,6.773956,6.362829,6.691099,6.708878,7.162359,...,6.170036,6.073339,6.091008,6.362465,5.255758,6.321685,6.347607,5.942381,3.489235,6.714704
3,0610009B22Rik,6.169217,6.473920,6.314496,6.533899,6.450335,6.585420,6.825147,6.757542,6.932700,...,6.695441,6.609076,6.495534,6.446457,6.901063,6.716616,6.559989,6.101363,2.445023,7.059285
4,0610009E02Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45764,n-R5s143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45765,n-R5s144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45766,n-R5s146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Take first 93-271 columns - those are all the glutamatergic cells

In [8]:
glut_cells_noname = list(rna_seq_df.columns)[93:271]
glut_cells = ['Unnamed: 0'] + glut_cells_noname
# add column with names of gene
excit_df = rna_seq_df[glut_cells]
excit_df

,Unnamed: 0,DG_1,DG_2,DG_3,RSP/ACA L4/5 IT_1,L2/3 IT Cxcl14_1,L2/3 IT Cxcl14_2,L2/3 IT Cxcl14_3,L2/3 IT Cxcl14_4,L2/3 IT Cxcl14_5,...,CA1sp_9,CA1sp_10,CA3sp_1,CA3sp_2,CA3sp_3,CA3sp_4,CA3sp_5,CA3sp_6,CA3sp_7,CA2sp/IG
0,0610005C13Rik,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0610006L08Rik,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0610007P14Rik,4.618636,7.062095,6.220234,4.20869,6.613267,6.885900,6.622375,6.173526,5.610042,...,6.672252,6.818466,6.713735,6.591830,6.917969,6.461046,6.949309,7.004281,6.708835,6.758041
3,0610009B22Rik,2.637047,5.964760,5.406703,0.00000,6.229554,6.253937,5.400589,6.092712,4.933765,...,6.390016,6.314109,6.053024,6.332248,6.185907,6.406308,4.970513,6.002995,6.127116,6.121678
4,0610009E02Rik,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45764,n-R5s143,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45765,n-R5s144,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45766,n-R5s146,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


see if the median is differen between excitiatryo and inhibitory
each column is a unique neuron
treat each column as a different trial as the cell
10 indiviidual t tests for reach gene

do t_test of each row - inhib vs exictaty

In [28]:
def ttest_stats(index_of_gene):
    '''
    returns ttest results of exitatory df and inhib df of a gene at a specific index
    inputs: index number of gene
    '''
    return stats.ttest_ind(excit_df.iloc[index_of_gene][1:], inhib_df.iloc[index_of_gene][1:])
    

In [30]:
for i in range(10):
    print(ttest_stats(i))

Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=2.2047547520692707, pvalue=0.028324193767009006)
Ttest_indResult(statistic=-4.170432087314668, pvalue=4.1138480272786444e-05)
Ttest_indResult(statistic=0.7143541196981078, pvalue=0.47563228815014913)
Ttest_indResult(statistic=-1.3754106286948975, pvalue=0.1701570171303062)
Ttest_indResult(statistic=1.1397449584339512, pvalue=0.25541399479236193)
Ttest_indResult(statistic=nan, pvalue=nan)
Ttest_indResult(statistic=3.6048746817164203, pvalue=0.000372537666184519)
Ttest_indResult(statistic=-1.5480589454655171, pvalue=0.12279255591210372)
